# Milestone 1

In this milestone, we will be **reading in vanilla CSV files via Pandas**. We will first download the data, retrieve unzip the folder then combine the individual CSV files into a combined file. After which, we will perform EDA on Python and R. 

The **objective for the Python aspect** is to determine which approach is better in loading the combined CSV file to save time and storage. 

The **objective for the R aspect** is to determine which approach is best to transfer the dataframe from Python to R. 

## 1. Downloading the data

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
# Necessary metadata
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfallNSW/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [5]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

## 2. Combining CSV files

In [6]:
%%time
import pandas as pd

path = r'rainfallNSW' # use your path
all_files = glob.glob(path + "/*.csv")
all_files.remove(path + "/observed_daily_rainfall_SYD.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['model'] = filename[12:-23]
    li.append(df)

combined_df = pd.concat(li, axis=0, ignore_index=True)
combined_df

CPU times: user 52.2 s, sys: 10.4 s, total: 1min 2s
Wall time: 1min 7s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
62467838,2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
62467839,2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
62467840,2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON
62467841,2014-12-30 12:00:00,-30.157068,-29.214660,153.1250,154.3750,8.541592e+00,SAM0-UNICON


### 2.1. Time taken to combine CSV file

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Ruben       |       MacOS      |  8  |    M1     |   Yes  |  50s       |
| Jacqueline  |       MacOS      |  8  |IntelCorei5|   Yes  |  1min 7s   |
| Kyle        |       Windows 10 |  16 |Intelcorei7|   Yes  |  2min 30s  |
| Sanchit     |       MacOS      |  8  |    M1     |   Yes  |  40s       |

### 2.2 Observations

We can see that both the **Operating System** and the **Processor** of the computer affect the amount of time it takes to combine the csv files.  The M1 chip performed better than the Intel processors, since it took less time to combine the csv files, and overall we can see that the MacOS operating system performed better than the Windows OS, in spite of the fact that the MacOS computers had 8GB of RAM vs 16 GB of RAM that the Windows computer has.  

Also, the Intel Mac computer had an i5 processor, and performed better than the Windows computer with an i7 processor, which shows the difference between the Operating Systems.  

However, theoretically, this believe that the difference should not be so stark since the computer has a much faster processing speed. This might be due to the individual's laptop. 

## 3. Python EDA 

We will try out various approaches to reduce run time and memory usage and performing a simple EDA for predicting the daily rainfall in Australia. 

We recorded our baseline, via **pd.read_csv**. Afterwhich, we will compare two approaches to the baseline.  
- The **first approach** is to change *dtype* of the data
- The **second approach** is to only load 1 column and see its effect on the run time.

We will run the above tasks and note down the run time in each members' computer. 

### 3.1 Baseline: Load in CSV file

**Saving combined csv**

In [7]:
combined_df.to_csv("combined_df.csv")

In [8]:
%%time
combined_df = pd.read_csv("combined_df.csv")
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   time           object 
 2   lat_min        float64
 3   lat_max        float64
 4   lon_min        float64
 5   lon_max        float64
 6   rain (mm/day)  float64
 7   model          object 
dtypes: float64(5), int64(1), object(2)
memory usage: 3.7+ GB
CPU times: user 1min 11s, sys: 44 s, total: 1min 54s
Wall time: 2min 5s


#### 3.1.1 Time taken to load the dataset

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Ruben       |       MacOS      |  8  |    M1     |   Yes  |  1min 36s  |
| Jacqueline  |       MacOS      |  8  |IntelCorei5|   Yes  |  2min 5s   |
| Kyle        |       Windows 10 |  16 |Intelcorei7|   Yes  |  4min 45s  |
| Sanchit     |       MacOS      |  8  |    M1     |   Yes  |  1min 32s  |

#### 3.1.2 Observations

Comparing between operating systems, it appears that while the Windows 10 OS had a higher RAM, the time taken to load the dataset is significantly slower (took twice to three times the duration to load). 

Comparing between the difference processors in MacOS, it is clear that to load the CSV file, the M1 processor performed about 25% faster than the IntelCorei5 processor.

### 3.2 Approach 1: Convert numerical columns from float64 to float32

In [9]:
%%time
combined_df_f32 = pd.read_csv("combined_df.csv", dtype={'lat_min': 'float32', 'lat_max': 'float32', 'lon_min': 'float32', 'lon_max': 'float32', 'rain (mm/day)': 'float32'})
combined_df_f32.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   time           object 
 2   lat_min        float32
 3   lat_max        float32
 4   lon_min        float32
 5   lon_max        float32
 6   rain (mm/day)  float32
 7   model          object 
dtypes: float32(5), int64(1), object(2)
memory usage: 2.6+ GB
CPU times: user 1min 11s, sys: 34 s, total: 1min 45s
Wall time: 1min 54s


#### 3.2.1 Time taken to load the dataset

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Ruben       |       MacOS      |  8  |    M1     |   Yes  | 1min 27s   |
| Jacqueline  |       MacOS      |  8  |IntelCorei5|   Yes  | 1min 54s    |
| Kyle        |       Windows 10 |  16 |Intelcorei7|   Yes  | 5min 16s  |
| Sanchit     |       MacOS      |  8  |    M1     |   Yes  | 1min 12s   |

#### 3.2.2 Observations

Overall, the total time taken to load the data did not reduce considerably, but the memory usage reduced from upwards of 3.7GB to upwards of 2.6GB.   

Comparing between the operating systems, the Windows10 OS took twice to three times longer to load the new CSV file compared to MacOS.  

Comparing between the different processors in MacOS, there is an interesting observation. For the IntelCorei5 processor, the time difference in loading the new CSV file is 9s, while the difference for the M1 chip is between 9 to 20s, which is much higher. This could be an instance where the new M1 chip works more effectively.

### 3.3 Approach 2: Load individual columns

In [10]:
%%time
column = ['lat_max']
combined_df_value = pd.read_csv("combined_df.csv", usecols=column)
print(combined_df_value.value_counts())

lat_max   
-31.099476    3035329
-32.041885    3035329
-34.869110    3035329
-32.984293    3035329
-30.000000    2529870
               ...   
-33.487232     183960
-33.490981     183960
-30.696652     183960
-30.700015     183960
-27.906064     183960
Length: 87, dtype: int64
CPU times: user 33.5 s, sys: 3.76 s, total: 37.2 s
Wall time: 38.6 s


#### 3.3.1 Time taken to load the dataset

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Ruben       |       MacOS      |  8  |    M1     |   Yes  |  29s       |
| Jacqueline  |       MacOS      |  8  |IntelCorei5|   Yes  |  38.6s     |
| Kyle        |       Windows 10 |  16 |Intelcorei7|   Yes  |  2min 36s  |
| Sanchit     |       MacOS      |  8  |    M1     |   Yes  |  26s       |

#### 3.3.2 Observations

Overall, loading only one column is significantly faster than loading in the entire data set. Compared to loading one column compared to 7 columns (baseline), it took one third the duration, regardless of computer. 

We will not be comparing between different OS as there is a recurrent theme that there might be something wrong with Kyle's computer. It might be due to hardware failure or that his hard drive may be full, but these are just speculations. 

Comparing between the different processors for the MacOS, the M1 chip performs at 1.5 times the speed compared to the IntelCorei5 chip.

**However, this is not a feasible solution as it is not scalable. Performing this task for all columns in a loop will only add to the total time.**

### 3.4 EDA using combined_df file

In [11]:
%%time
combined_df.describe()

CPU times: user 13.5 s, sys: 9.43 s, total: 22.9 s
Wall time: 26.9 s


,Unnamed: 0,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,3.123392e+07,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.803291e+07,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,0.000000e+00,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,1.561696e+07,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,3.123392e+07,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,4.685088e+07,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,6.246784e+07,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


#### 3.4.1 Time taken to run .describe()

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Ruben       |       MacOS      |  8  |    M1     |   Yes  |  13s       |
| Jacqueline  |       MacOS      |  8  |IntelCorei5|   Yes  |  26.9s       |
| Kyle        |       Windows 10 |  16 |Intelcorei7|   Yes  |  35s  |
| Sanchit     |       MacOS      |  8  |    M1     |   Yes  |  14s       |

#### 3.4.2 Observations

This is a clear instance where the processor made a significant difference. The M1 chip compared to the i5/i7 chip took less than half the time to run this line of code. 



## 4. R EDA

### 4.1 Parquet file

In [12]:
%load_ext rpy2.ipython

In [13]:
%%time
combined_df.to_parquet("rainfallNSW/combined_data.parquet")

CPU times: user 24.2 s, sys: 20.2 s, total: 44.4 s
Wall time: 55.5 s


In [14]:
%%R
suppressMessages(library(data.table))
suppressMessages(library(dplyr, warn.conflicts = FALSE))
suppressMessages(library(arrow, warn.conflicts = FALSE))

In [15]:
%%R
r_parquet <- open_dataset("rainfallNSW/combined_data.parquet")
r_df <- r_parquet |> collect()

> We decided to use a Parquet file to transfer our data frame for several reasons.
> 1. **Time:** This approach allows for the quickest and most convenient recovery in the event that we have to restart the kernel or the notebook crashes. Essentially this us to pick up where we left off without having to repeat the earlier steps in python.
> 2. **Flexibility**: Parquet allows us to easily read our data into many different languages without needing to know the interactions between said languages. Furthermore we can save time and memory by using partitioning to read in only what is needed, rather than loading the entire data frame into memory and then filtering it.
> 3. **Memory**: Having consolidated our data into a parquet file we no longer need to keep our original CSV files. This essentially allows us to store the same information with a fraction of the memory requirement, in a format that is arguably more desirable than CSV.
>  
> - We did experiment with Arrow Exchange before ultimately deciding to use Parquet. Arrow Exchange code is commented out at the bottom of the notebook

#### 4.1.1 EDA using Parquet file

In [16]:
%%time
%%R
r_df |> str()

tibble [62,467,843 × 8] (S3: tbl_df/tbl/data.frame)
 $ Unnamed: 0   : int [1:62467843] 0 1 2 3 4 5 6 7 8 9 ...
 $ time         : chr [1:62467843] "1889-01-01 12:00:00" "1889-01-02 12:00:00" "1889-01-03 12:00:00" "1889-01-04 12:00:00" ...
 $ lat_min      : num [1:62467843] -35.4 -35.4 -35.4 -35.4 -35.4 ...
 $ lat_max      : num [1:62467843] -33.6 -33.6 -33.6 -33.6 -33.6 ...
 $ lon_min      : num [1:62467843] 142 142 142 142 142 ...
 $ lon_max      : num [1:62467843] 143 143 143 143 143 ...
 $ rain (mm/day): num [1:62467843] 4.24e-13 4.22e-13 4.50e-13 4.25e-13 4.27e-13 ...
 $ model        : chr [1:62467843] "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" ...
CPU times: user 93.4 ms, sys: 38.2 ms, total: 132 ms
Wall time: 256 ms


In [17]:
%%time
%%R
r_df |> head()

# A tibble: 6 × 8
  `Unnamed: 0` time        lat_min lat_max lon_min lon_max `rain (mm/day)` model
         <int> <chr>         <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>
1            0 1889-01-01…   -35.4   -33.6    142.    143.        4.24e-13 MPI-…
2            1 1889-01-02…   -35.4   -33.6    142.    143.        4.22e-13 MPI-…
3            2 1889-01-03…   -35.4   -33.6    142.    143.        4.50e-13 MPI-…
4            3 1889-01-04…   -35.4   -33.6    142.    143.        4.25e-13 MPI-…
5            4 1889-01-05…   -35.4   -33.6    142.    143.        4.27e-13 MPI-…
6            5 1889-01-06…   -35.4   -33.6    142.    143.        4.20e-13 MPI-…
CPU times: user 13.2 s, sys: 7.32 s, total: 20.5 s
Wall time: 24 s


In [18]:
%%time
%%R
r_df |> summary()

   Unnamed: 0           time              lat_min           lat_max      
 Min.   :       0   Length:62467843    Min.   :-36       Min.   :-36.00  
 1st Qu.:15616960   Class :character   1st Qu.:-35       1st Qu.:-33.66  
 Median :31233921   Mode  :character   Median :-33       Median :-32.04  
 Mean   :31233921                      Mean   :-33       Mean   :-31.98  
 3rd Qu.:46850882                      3rd Qu.:-31       3rd Qu.:-30.16  
 Max.   :62467842                      Max.   :-30       Max.   :-27.91  
                                       NA's   :3219300                   
    lon_min           lon_max      rain (mm/day)        model          
 Min.   :141       Min.   :141.2   Min.   :  0       Length:62467843   
 1st Qu.:143       1st Qu.:145.0   1st Qu.:  0       Class :character  
 Median :147       Median :148.1   Median :  0       Mode  :character  
 Mean   :147       Mean   :148.2   Mean   :  2                         
 3rd Qu.:150       3rd Qu.:151.3   3rd Qu.:  1  

### 4.2 Arrow Exchange

In [19]:
#import pyarrow as pa
#import pyarrow 
#from pyarrow import csv
#import rpy2_arrow.pyarrow_rarrow as pyra

In [20]:
#pyarrow_table = pa.Table.from_pandas(combined_df)
#r_table = pyra.converter.py2rpy(pyarrow_table)

In [21]:
#%%time
#%%R -i r_table
#suppressMessages(library(arrow, warn.conflicts = FALSE))
#suppressMessages(library(dplyr, warn.conflicts = FALSE))
#r_df <- r_table |> collect()